
# Overview
This CodeLab demonstrates how to build a fused TFLite LSTM model for MNIST recognition using Keras, and how to convert it to TensorFlow Lite.

The CodeLab is very similar to the Keras LSTM [CodeLab](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/examples/experimental_new_converter/keras_lstm.ipynb). However, we're creating fused LSTM ops rather than the unfused versoin.

Also note: We're not trying to build the model to be a real world application, but only demonstrate how to use TensorFlow Lite. You can a build a much better model using CNN models. For a more canonical lstm codelab, please see [here](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py).


# Step 0: Prerequisites
It's recommended to try this feature with the newest TensorFlow nightly pip build.

In [ ]:
!pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 584.5 MB 4.5 kB/s 
     |████████████████████████████████| 5.9 MB 35.4 MB/s 
     |████████████████████████████████| 1.7 MB 56.1 MB/s 
     |████████████████████████████████| 439 kB 49.0 MB/s 
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3


# Step 1: Build the MNIST LSTM model.

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
from keras.layers.rnn.dropout_rnn_cell_mixin import DropoutRNNCellMixin
from tensorflow.python.ops.nn_ops import dropout
from tensorflow.python.ops.gen_nn_ops import relu

actions = [
    'left',
    'right',
    'hi',
    'look'
    
]

data = np.concatenate([
    
    np.load('seq_hi_1_1661663755.npy'),
    np.load('seq_hi_2_1661663809.npy'),
    np.load('seq_hi_3_1661663843.npy'),
    np.load('seq_hi_4_1661663887.npy'),
    np.load('seq_eat_1_1661663981.npy'),
    np.load('seq_eat_2_1661664034.npy'),
    np.load('seq_eat_3_1661664070.npy'),
    np.load('seq_eat_4_1661664107.npy'),
    np.load('seq_bob_1_1661664188.npy'),
    np.load('seq_bob_2_1661664238.npy'),
    np.load('seq_bob_3_1661664273.npy'),
    np.load('seq_bob_4_1661664322.npy'),
    np.load('seq_meet_1_1661664416.npy'),
    np.load('seq_meet_2_1661664458.npy'),
    np.load('seq_meet_3_1661664486.npy'),
    np.load('seq_meet_4_1661664522.npy')
    # np.load('keras_lstm/seq_look_2_1660103472.npy'),
    # np.load('keras_lstm/seq_look_4_1660105439.npy'),
    # np.load('keras_lstm/seq_look_5_1660105557.npy'),
    # np.load('keras_lstm/seq_look_6_1660105715.npy'),
    # np.load('keras_lstm/seq_look_7_1660105752.npy'),
    # np.load('keras_lstm/seq_look_8_1660105786.npy'),
    # np.load('keras_lstm/seq_look_9_1660105825.npy'),
    # np.load('keras_lstm/seq_look_10_1660105849.npy'),
    # np.load('keras_lstm/seq_look_12_1660105907.npy'),
    # np.load('keras_lstm/seq_look_13_1660105955.npy'),
    # np.load('keras_lstm/seq_look_14_1660106613.npy'),
    # np.load('keras_lstm/seq_look_15_1660106681.npy'),
    # np.load('keras_lstm/seq_look_16_1660107168.npy'),
    # np.load('keras_lstm/seq_look_17_1660107203.npy')
    
    
], axis=0)   
data.shape

data2 = np.concatenate([
    
    np.load('seq_hi_5_1661663923.npy'),
    np.load('seq_eat_5_1661664150.npy'),
    np.load('seq_bob_5_1661664366.npy'),
    np.load('seq_meet_5_1661664574.npy')
    
    
], axis=0)  
data.shape

x_data = data[:,:,:-1]
x_data2 = data2[:,:,:-1]
labels = data[:,0,-1]
labels2 = data2[:, 0, -1]
y_data=tf.keras.utils.to_categorical(labels, num_classes=len(actions))

y_data2 =tf.keras.utils.to_categorical(labels2, num_classes=len(actions))

x_data = x_data.astype(np.float32)
y_data = labels.astype(np.float32)

x_data2 = x_data2.astype(np.float32)
y_data2 = y_data2.astype(np.float32)

x_train = x_data


x_val = x_data2
y_train = y_data

y_val = y_data2


# model2 = tf.keras.models.Sequential([
#    tf.keras.layers.Input(shape=(30,432),name='input'),
#    tf.keras.layers.LSTM(20, time_major=False, return_sequences=True),
#    tf.keras.layers.Flatten(),
#    tf.keras.layers.Dense(3, activation=tf.nn.softmax, name='output')
# ])

model2 = tf.keras.models.Sequential([
   tf.keras.layers.Input(shape=(30,368),name='input'),
   tf.keras.layers.LSTM(64, time_major=False, return_sequences=True),
   tf.keras.layers.Dropout(0.3),
   tf.keras.layers.Dense(32, activation=tf.nn.relu),
   tf.keras.layers.Dense(32, activation=tf.nn.relu),
   tf.keras.layers.Dropout(0.3),
   tf.keras.layers.Dropout(0.3),
   tf.keras.layers.Dense(32, activation=tf.nn.relu),
   tf.keras.layers.Flatten(),
   tf.keras.layers.Dense(4, activation=tf.nn.softmax, name='output')
])
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.summary()

_EPOCHS = 200


model2.fit(x_train, labels, epochs=_EPOCHS)


IndexError: ignored

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(28, 28), name='input'),
    tf.keras.layers.LSTM(20, time_major=False, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax, name='output')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 28, 20)            3920      
                                                                 
 flatten_2 (Flatten)         (None, 560)               0         
                                                                 
 output (Dense)              (None, 10)                5610      
                                                                 
Total params: 9,530
Trainable params: 9,530
Non-trainable params: 0
_________________________________________________________________


# Step 2: Train & Evaluate the model.
We will train the model using MNIST data.

In [ ]:
# Load MNIST dataset.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

# Change this to True if you want to test the flow rapidly.
# Train with a small dataset and only 1 epoch. The model will work poorly
# but this provides a fast way to test if the conversion works end to end.
_FAST_TRAINING = False
_EPOCHS = 5
if _FAST_TRAINING:
  _EPOCHS = 1
  _TRAINING_DATA_COUNT = 1000
  x_train = x_train[:_TRAINING_DATA_COUNT]
  y_train = y_train[:_TRAINING_DATA_COUNT]


model.fit(x_train, y_train, epochs=_EPOCHS)
model.evaluate(x_test, y_test, verbose=0)

Epoch 1/5
1875/1875 [==============================] - 23s 11ms/step - loss: 0.3244 - accuracy: 0.9038
Epoch 2/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.1193 - accuracy: 0.9645
Epoch 3/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0886 - accuracy: 0.9728
Epoch 4/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.0729 - accuracy: 0.9776
Epoch 5/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0624 - accuracy: 0.9811


[0.058012884110212326, 0.9817000031471252]

# Step 3: Convert the Keras model to TensorFlow Lite model.

In [ ]:
run_model = tf.function(lambda x: model2(x))
# This is important, let's fix the input size.
BATCH_SIZE = 1
STEPS = 30
INPUT_SIZE = 368
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model2.inputs[0].dtype))

# model directory.
MODEL_DIR = "AAA"
model2.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()
open("AAAA.tflite", "wb").write(tflite_model)

541976

In [ ]:
run_model = tf.function(lambda x: model(x))
# This is important, let's fix the input size.
BATCH_SIZE = 1
STEPS = 28
INPUT_SIZE = 28
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model.inputs[0].dtype))

# model directory.
MODEL_DIR = "keras_lstm"
model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
tflite_model = converter.convert()
open("BBBB.tflite", "wb").write(tflite_model)

41220

# Step 4: Check the converted TensorFlow Lite model.
Now load the TensorFlow Lite model and use the TensorFlow Lite python interpreter to verify the results.

In [ ]:
# Run the model with TensorFlow to get expected results.
TEST_CASES = 10

# Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)
for i in range(TEST_CASES):
  expected = model.predict(x_test[i:i+1])
  interpreter.set_tensor(input_details[0]["index"], x_test[i:i+1, :, :])
  interpreter.invoke()
  result = interpreter.get_tensor(output_details[0]["index"])

  # Assert if the result of TFLite model is consistent with the TF model.
  np.testing.assert_almost_equal(expected, result, decimal=5)
  print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

  # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
  # the states.
  # Clean up internal states.
  interpreter.reset_all_variables()

[{'name': 'serving_default_x:0', 'index': 0, 'shape': array([ 1, 28, 28], dtype=int32), 'shape_signature': array([ 1, 28, 28], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 21, 'shape': array([ 1, 10], dtype=int32), 'shape_signature': array([ 1, 10], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
1/1 [==============================] - 0s 55ms/step
Done. The result of TensorFlow matches the result of TensorFlow Lite.
1/1 [==============================] - 0s 38ms/step
Done. The result of TensorFlow matches the result of TensorFlow Lite.
1/1 [==============================] - 

In [ ]:
# Run the model with TensorFlow to get expected results.
TEST_CASES = 10

# Run the model with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)
for i in range(TEST_CASES):
  expected = model2.predict(x_val[i:i+1])
  interpreter.set_tensor(input_details[0]["index"], x_val[i:i+1, :, :])
  interpreter.invoke()
  result = interpreter.get_tensor(output_details[0]["index"])

  # Assert if the result of TFLite model is consistent with the TF model.
  np.testing.assert_almost_equal(expected, result, decimal=5)
  print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

  # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
  # the states.
  # Clean up internal states.
  interpreter.reset_all_variables()

[{'name': 'serving_default_x:0', 'index': 0, 'shape': array([  1,  30, 432], dtype=int32), 'shape_signature': array([ -1,  30, 432], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 29, 'shape': array([1, 3], dtype=int32), 'shape_signature': array([-1,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
1/1 [==============================] - 0s 22ms/step
Done. The result of TensorFlow matches the result of TensorFlow Lite.
1/1 [==============================] - 0s 26ms/step
Done. The result of TensorFlow matches the result of TensorFlow Lite.
1/1 [==============================

# Step 5: Let's inspect the converted TFLite model.

Let's check the model, you can see the LSTM will be in it's fused format.

![Fused LSTM](https://raw.githubusercontent.com/tensorflow/tensorflow/master/tensorflow/lite/examples/experimental_new_converter/keras_lstm.png)
